In [1]:
import sys,os
sys.path.append('../../')
import numpy as np
from smodels.base import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.decomposition import decomposer
from smodels.base.physicsUnits import fb, GeV, pb
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.matching.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.base.smodelsLogging import setLogLevel
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
setLogLevel("error")

# Set the path to the database
database = Database('unittest')

### Load model

In [2]:
slhafile = '../../inputFiles/slha/longLived.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 0.01*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

### Get experimental results

In [4]:
listOfExpRes = database.expResultList

### Compute the theory predictions

In [5]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = theoryPredictionsFor(database, topDict, combinedResults=False)
predsDict = {}
for tp in allPredictions:
    anaID = tp.analysisId()
    if anaID not in predsDict:
        predsDict[anaID] = []
    predsDict[anaID].append(tp)


for anaID,predictions in predsDict.items():
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % anaID)
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset
        datasetID = theoryPrediction.dataId()
        txnames = sorted([str(txname) for txname in theoryPrediction.txnames])
        print("------------------------")
        print("Dataset = ", datasetID)  # Analysis name
        print("TxNames = ", txnames)
        print("Theory Prediction = ", theoryPrediction.xsection)  # Signal cross section
        if isinstance(theoryPrediction.conditions,list):
            cond = ['%1.2E' %c for c in theoryPrediction.conditions]
        else:
            cond = theoryPrediction.conditions
        print("Condition Violation = ", cond)  # Condition violation values

        # Get the corresponding upper limit:
        print("UL for theory prediction = ", theoryPrediction.upperLimit)

        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.1E" % r)
        # Compute likelihoods for EM-type results:
        if dataset.getType() == 'efficiencyMap':
            theoryPrediction.computeStatistics()
            print('L_BSM, L_SM, L_max = %1.1E, %1.1E, %1.1E' % (theoryPrediction.likelihood(),
                  theoryPrediction.lsm(), theoryPrediction.lmax()))
        if r > rmax:
            rmax = r
            bestResult = anaID
        allPredictions.append(theoryPrediction)


 Theory Predictions and Constraints:

 CMS-EXO-13-006 
------------------------
Dataset =  c200track
TxNames =  ['THSCPM1', 'THSCPM2', 'THSCPM3', 'THSCPM4', 'THSCPM5', 'THSCPM6', 'THSCPM7']
Theory Prediction =  3.70E-04 [pb]
Condition Violation =  None
UL for theory prediction =  1.60E-01 [fb]
r = 2.3E+00
L_BSM, L_SM, L_max = 2.0E-03, 2.1E+00, 2.1E+00

 CMS-SUS-16-039 
------------------------
Dataset =  None
TxNames =  ['TChiChipmSlepL']
Theory Prediction =  1.69E-03 [pb]
Condition Violation =  ['2.03E-01', '0.00E+00', '0.00E+00']
UL for theory prediction =  5.70E+00 [fb]
r = 3.0E-01
------------------------
Dataset =  None
TxNames =  ['TChiChipmSlepStau']
Theory Prediction =  5.94E-05 [pb]
Condition Violation =  ['0.00E+00', '0.00E+00']
UL for theory prediction =  5.06E+00 [fb]
r = 1.2E-02

 CMS-SUS-16-050-agg 
------------------------
Dataset =  ar6
TxNames =  ['T2tt']
Theory Prediction =  2.75E-04 [pb]
Condition Violation =  None
UL for theory prediction =  2.87E+00 [fb]
r = 9.6E-